# Exploring Hacker News Post

In this project, we'll work with a data set of submissions to popular technology site Hacker News.

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

You can find the data set here, but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are descriptions of the columns:

- id: The unique identifier from Hacker News for the post
- title: The title of the post
- url: The URL that the posts links to, if the post has a URL
- num_points: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
- num_comments: The number of comments that were made on the post
- author: The username of the person who submitted the post
- created_at: The date and time at which the post was submitted

***
We're specifically interested in posts whose titles begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question. Below are a couple examples:

> Ask HN: How to improve my personal website?<br>
Ask HN: Am I the only one outraged by Twitter shutting down share counts?<br>
Ask HN: Aby recent changes to CSS that broke mobile?

Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting. Below are a couple of examples:

> Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform'<br>
Show HN: Something pointless I made<br>
Show HN: Shanhu.io, a programming playground powered by e8vm<br>

We'll compare these two types of posts to determine the following:

Do Ask HN or Show HN receive more comments on average?
Do posts created at a certain time receive more comments on average?

# Read in the data

In [3]:
from csv import reader
opened_file = open('HN_posts_year_to_Sep_26_2016.csv', encoding="utf8")

read_file = reader(opened_file)
hn_data = list(read_file)
headers = hn_data[0]
hn = hn_data[1:]

print(headers)
print('\n')
print(hn[:4])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26'], ['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24'], ['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19'], ['12578989', 'algorithmic music', 'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext', '1', '0', 'poindontcare', '9/26/2016 3:16']]


# Extracting the ASK HN and SHOW HN posts

In [9]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

print('ask_posts length: ', len(ask_posts))
print('show_posts_length: ', len(show_posts))
print('other_posts_length: ', len(other_posts))


ask_posts length:  9139
show_posts_length:  10158
other_posts_length:  273822
[['12578908', 'Ask HN: What TLD do you use for local development?', '', '4', '7', 'Sevrene', '9/26/2016 2:53'], ['12578522', 'Ask HN: How do you pass on your work when you die?', '', '6', '3', 'PascLeRasc', '9/26/2016 1:17'], ['12577908', 'Ask HN: How a DNS problem can be limited to a geographic region?', '', '1', '0', 'kuon', '9/25/2016 22:57'], ['12577870', 'Ask HN: Why join a fund when you can be an angel?', '', '1', '3', 'anthony_james', '9/25/2016 22:48'], ['12577647', 'Ask HN: Someone uses stock trading as passive income?', '', '5', '2', '00taffe', '9/25/2016 21:50']]


# Calculating the Average Number of Comments for Ask HN and Show HN Posts

In [14]:
def count_comments(posts, index):
    total_comments = 0
    for row in posts:
        total_comments += int(row[index])
    avg_comments = total_comments / len(posts)
    return total_comments, avg_comments

total_ask_comments, avg_ask_comments = count_comments(ask_posts, 4)
print('total_ask_comments: ', total_ask_comments)
print('avg_ask_comments: ', avg_ask_comments)

total_show_comments, avg_show_comments = count_comments(show_posts, 4)
print('\n')
print('total_show_comments: ', total_show_comments)
print('avg_show_comments: ', avg_show_comments)
        

total_ask_comments:  94986
avg_ask_comments:  10.393478498741656


total_show_comments:  49633
avg_show_comments:  4.886099625910612


On average, ask posts in our sample receive approximately 10 comments, whereas show posts receive approximately 4. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

# Finding the Amount of Ask Posts and Comments by Hour Created

In [21]:
import datetime as dt

def posts_created_by_hour(posts, index_1, index_2):
    counts_by_hour = {}
    comments_by_hour = {}
    for row in posts:
        to_date = dt.datetime.strptime(row[index_1],  "%m/%d/%Y %H:%M")
        hour = to_date.strftime('%H')
        num_comments = int(row[index_2])
        
        if hour not in counts_by_hour:
            counts_by_hour[hour] = 1
            comments_by_hour[hour] = num_comments
        else:
            counts_by_hour[hour] += 1
            comments_by_hour[hour] += num_comments
    return counts_by_hour, comments_by_hour

    
ask_counts_by_hour,ask_comments_by_hour = posts_created_by_hour(ask_posts, 6, 4)

print('ask_counts_by_hour: ', ask_counts_by_hour)
print('\n')
print('ask_comments_by_hour: ', ask_comments_by_hour)

ask_counts_by_hour:  {'02': 269, '01': 282, '22': 383, '21': 518, '19': 552, '17': 587, '15': 646, '14': 513, '13': 444, '11': 312, '10': 282, '09': 222, '07': 226, '03': 271, '23': 343, '20': 510, '16': 579, '08': 257, '00': 301, '18': 614, '12': 342, '04': 243, '06': 234, '05': 209}


ask_comments_by_hour:  {'02': 2996, '01': 2089, '22': 3372, '21': 4500, '19': 3954, '17': 5547, '15': 18525, '14': 4972, '13': 7245, '11': 2797, '10': 3013, '09': 1477, '07': 1585, '03': 2154, '23': 2297, '20': 4462, '16': 4466, '08': 2362, '00': 2277, '18': 4877, '12': 4234, '04': 2360, '06': 1587, '05': 1838}


# Calculating the Average Number of Comments for Ask HN Posts by Hour

In [28]:
def avg_comments_by_hour(hour_dict, comments_dict):
    avg_by_hour = []
    for key in hour_dict:
        avg_by_hour.append([key, comments_dict[key]/hour_dict[key]])
    return avg_by_hour

ask_avg_by_hour = avg_comments_by_hour(ask_counts_by_hour,ask_comments_by_hour)
ask_avg_by_hour_sorted = sorted(ask_avg_by_hour,  reverse=True, key=lambda x: x[1])
print('ask_avg_by_hour_sorted: ', ask_avg_by_hour_sorted)

print('\n')
print("Top 5 hours for 'Ash HN' Comments")
for hrs, avg in ask_avg_by_hour_sorted[:5]:
    print('{hour}: {avg:.2f} avgerage comments per post'.format(hour=dt.datetime.strptime(hrs, '%H').strftime('%H:%M'), avg=avg))

ask_avg_by_hour_sorted:  [['15', 28.676470588235293], ['13', 16.31756756756757], ['12', 12.380116959064328], ['02', 11.137546468401487], ['10', 10.684397163120567], ['04', 9.7119341563786], ['14', 9.692007797270955], ['17', 9.449744463373083], ['08', 9.190661478599221], ['11', 8.96474358974359], ['22', 8.804177545691905], ['05', 8.794258373205741], ['20', 8.749019607843136], ['21', 8.687258687258687], ['03', 7.948339483394834], ['18', 7.94299674267101], ['16', 7.713298791018998], ['00', 7.5647840531561465], ['01', 7.407801418439717], ['19', 7.163043478260869], ['07', 7.013274336283186], ['06', 6.782051282051282], ['23', 6.696793002915452], ['09', 6.653153153153153]]


Top 5 hours for 'Ash HN' Comments
15:00: 28.68 avgerage comments per post
13:00: 16.32 avgerage comments per post
12:00: 12.38 avgerage comments per post
02:00: 11.14 avgerage comments per post
10:00: 10.68 avgerage comments per post


# Conclusion

In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as ask post and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).

However, it should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, ask posts received more comments on average and ask posts created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments on average.